In [1]:
### Saurabh - 30Aug 2023 - Adhoc notebook for PA  
#### saurabhmangal@google.com

## V2- New updates on 30Aug -made the prompts works by adding token length and using prompting

## PA create JSON object for sending to SOLR 

In [88]:
from langchain import PromptTemplate, LLMChain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import VertexAI

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI(temperature=0.2,model_name='text-bison@latest', #model_name='text-bison@001'
    max_output_tokens=256,
    top_p=0.8,
    top_k=40,
    verbose=True,
              )

llm_chain = LLMChain(prompt=prompt, llm=llm)



In [89]:
import pandas as pd
import pprint

columns = ['FacType', 'Area', 'Location']
# Follow are the FacLoc.CSV raw file I've uploaded to Github for your convenience. The token will expire 7 days from 14 Sep!
#sUrlOrFilename = 'https://raw.githubusercontent.com/yongsh888/PA-POC/main/FacLoc.CSV?token=GHSAT0AAAAAACHLXIEU5ZVWV5OXVA5PZZGGZICNQOQ'
sUrlOrFilename = 'FacLoc.CSV'
dfFacility = pd.read_csv(sUrlOrFilename, header=None, names=columns)
pp = pprint.PrettyPrinter()
sFacilityTable = pp.pformat(dfFacility)
print(sFacilityTable)

               FacType        Area              Location
0         Tennis Court     Kallang       Kampong Glam CC
1         Tennis Court    Tampines      Our Tampines Hub
2     Badminton Courts  Ang Mo Kio         Kebun Baru CC
3     Badminton Courts  Ang Mo Kio          Teck Ghee CC
4     Badminton Courts  Ang Mo Kio            Thomson CC
..                 ...         ...                   ...
111  Table Tennis Room   Toa Payoh  Toa Payoh Central CC
112  Table Tennis Room   Toa Payoh    Toa Payoh South CC
113  Table Tennis Room   Woodlands      ACE The Place CC
114  Table Tennis Room      Yishun         Chong Pang CC
115       Soccer Field    Tampines      Our Tampines Hub

[116 rows x 3 columns]


### Step01: Convert NLP to JSON request

In [104]:
import datetime

question = """Find a badminton court near Bedok next Thur-Fri in the evening"""

# Set today's date to script execution day otherwise 9 Mar 2023 will be used.
prompt_01 =  "Today is " + datetime.datetime.now().isoformat() + ".\n"
prompt_01 =  prompt_01 + """

Convert this question to fill the following fields below. Only return the JSON part: 

Question : """ + question +  """
Fields : 
{
  sports:  example can be badminton, tennis, basketball, table tennis, soccer
  map: location examples can be Bedok, Ang Mo Kio, Bishan, Alexandra
  start date: date in yyyy-mm-dd format
  end date: date in yyyy-mm-dd format
  Time : can be all, morning, afternoon or evening
}
"""

# Following doesn't work too well ("badminton court" - no trailing s):
# facility: facility correspond to sports, e.g. badminton courts, tennis court, basketball court, table tennis room and soccer field


In [105]:
import json
import pprint

response_01 = llm_chain.run(prompt_01)
#print("response_01 :: \n",response_01)
res = json.loads(response_01)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(res)
#print(type(response_01))

response_01 :: 
  {
  "sports": "badminton",
  "map": "Bedok",
  "start date": "2023-09-28",
  "end date": "2023-09-29",
  "Time": "evening"
}
{   'Time': 'evening',
    'end date': '2023-09-29',
    'map': 'Bedok',
    'sports': 'badminton',
    'start date': '2023-09-28'}


### Step02

In [108]:
prompt_02 = '''Generate a JSON object for a badminton court search request with the following parameters:
 ''' + respose_01 + ''' The JSON object should have the following structure:

json
{
  "success": false,
  "message": null,
  "data": {
    "results": [],
    "actions": null,
    "slots": null,
    "errorMsg": null,
    "labels": null,
    "categories": null,
    "outlets": null,
    "categoryList": null,
    "date": "2023-08-30T00:00:00",
    "facilityTimeSetting": {
      "dayRange": 0,
      "showNextDayFlag": false
    },
    "page": 12,
    "totalResults": 84,
    "message": "",
    "query": "BADMINTON COURTS",
    "category": "Facility",
    "map": "Bedok"
  }
}


Please note that the `results` field should be empty, as this is a search request and the results have not yet been 
retrieved.
''' 




In [109]:
# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

respose_02 = llm_chain.run(prompt_02)
print("respose_02 :: \n",respose_02)

respose_02 :: 
  {
  "success": false,
  "message": null,
  "data": {
    "results": [],
    "actions": null,
    "slots": null,
    "errorMsg": null,
    "labels": null,
    "categories": null,
    "outlets": null,
    "categoryList": null,
    "date": "2023-08-30T00:00:00",
    "facilityTimeSetting": {
      "dayRange": 0,
      "showNextDayFlag": false
    },
    "page": 12,
    "totalResults": 84,
    "message": "",
    "query": "BADMINTON COURTS",
    "category": "Facility",
    "map": "Bedok"
  }
}


## Example 02